#  This notebook will be used for a capstone project

In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')
t = time.strftime(('%Y%m%d'))

# scraping Foursquare

In [2]:
CLIENT_ID = 'JZRNXPQIXMR2FEEA2DBGRYXGOVD2XCDT2KLOSYOQZI5SFYGH' # your Foursquare ID
CLIENT_SECRET = 'V3B1S3DCXG52PCVEY5KFHFO5S2BWYSCQQW5A045QGRT4ZXST' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JZRNXPQIXMR2FEEA2DBGRYXGOVD2XCDT2KLOSYOQZI5SFYGH
CLIENT_SECRET:V3B1S3DCXG52PCVEY5KFHFO5S2BWYSCQQW5A045QGRT4ZXST


In [3]:
from geopy.geocoders import Nominatim
address = 'Montreal'
locator = Nominatim(user_agent = 'cap')
lat,lng = locator.geocode(address).latitude,locator.geocode(address).longitude

In [4]:
u = f"https://api.foursquare.com/v2/venues/search?near=Montreal,Montreal&client_id={CLIENT_ID}\
&client_secret={CLIENT_SECRET}&v={t}&limit={5000}&radius={100000}"
rev = requests.get(u).json()

In [5]:
radius = 10000
url_base = "https://api.foursquare.com/v2/"
url = url_base + f"venues/explore?client_secret={CLIENT_SECRET}&client_id={CLIENT_ID}&ll={lat},{lng}&radius={radius}&limit={5000}&v={t}"


recomendations = requests.get(url).json()

In [6]:
radius = 10000
url_base = "https://api.foursquare.com/v2/"
url = url_base + f"venues/search?client_secret={CLIENT_SECRET}&client_id={CLIENT_ID}&ll={lat},{lng}&radius={radius}&limit={5000}&v={t}"


results = requests.get(url).json()

# cleaning the data

In [7]:
from pandas import json_normalize
rec = json_normalize(recomendations['response']['groups'][0]['items'])
dropping_list = ['referralId','reasons.count','reasons.items',
               'venue.location.labeledLatLngs','venue.location.cc','venue.location.city',
               'venue.location.state','venue.location.country','venue.location.formattedAddress',
               'venue.photos.count','venue.photos.groups',
               'venue.location.postalCode','venue.location.crossStreet','venue.location.neighborhood'
                ,'venue.location.distance','venue.venuePage.id']
rec = rec.drop(dropping_list,axis = 1)
rec['categories'] = rec['venue.categories'].map(lambda x : x[0]['name'])
rec = rec.drop('venue.categories', axis =1)
dropping_cat =  ["Pool" , "Pool Hall" , "Bar" , "Beer Garden" , "Men's Store" , "Nightclub" ]
for categorie in dropping_cat:
    rec = rec[rec['categories'] != categorie]

search = json_normalize(results['response']['venues'])
dropping_list = ['referralId','hasPerk','location.labeledLatLngs','location.cc',
                'location.city','location.state','location.country','location.formattedAddress',
                'location.crossStreet','location.postalCode','location.distance','venuePage.id']
search = search.drop(dropping_list, axis =1)
def f(x):
    if type(x) == type([]) and len(x) >=1:
        return x[0]['name']
search['categories'] = search['categories'].map(lambda x : f(x) )

dl = ['referralId','hasPerk','location.labeledLatLngs','location.cc','location.city','location.state',
     'location.country','location.formattedAddress','location.crossStreet','location.postalCode','location.postalCode',
     'venuePage.id']
rev_df = json_normalize(rev['response']['venues'])
rev_df.drop(dl, axis =1 , inplace = True)
rev_df['categories'] = rev_df['categories'].map(lambda x : f(x) )
rev_df = rev_df.rename({'location.lat':'lat',
              'location.lng' : 'lng',
            'location.address':'address'}, axis = 1)
search = search.rename({'location.lat':'lat',
              'location.lng' : 'lng',
            'location.address':'address'}, axis = 1)
rec = rec.rename({'venue.id':'id',
           'venue.name':'name',
           'venue.location.lat':'lat',
           'venue.location.lng' : 'lng',
            'venue.location.address':'address'}, axis =1)

# wanted to take ratings but I exceeded the quota

In [8]:
def like_(df,col='id'):
    df['like'] = np.nan
    for c,i in enumerate(df[col]):
        venue_id = i # ID of Ecco
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
        result = requests.get(url).json()
        try:
            df.loc[c,'like'] = result['response']['venue']['likes']['count']
        except:
            continue

# merging the data Frames

In [9]:
df = rev_df.append(rec).append(search)

In [10]:
df = df.drop_duplicates('id')

In [11]:
df.head(5)

,id,name,categories,lat,lng,address
0,5058b1a0e4b008a6a954b455,Valtra,EV Charging Station,43.228536,2.114476,NaN
1,55aa5627498e9265d39d301f,Camellas Lloret Maison d'hotes,Bed & Breakfast,43.199272,2.141256,4 Rue De L'Angle
2,4ecca289754ac92c121e4d43,Pont-canal de Rébenty,Bridge,43.244132,2.156265,La Métairie Grande
3,4ff98791e4b0463f723caa67,Aire de Montréal,Rest Area,43.228609,2.114393,Autoroute de Deux Mers
4,59a2b7cc9ec3995174eaf2a2,Total,Gas Station,43.200341,2.140633,Avenue du Lauragais


# cleaning the addresses

In [12]:
dummy = pd.get_dummies(df[['categories']], prefix = '',prefix_sep = '')
dummy['address'] = df['address']
dummy['address'] = dummy['address'].str.lower()
dummy['address'] = dummy['address'].str.replace(r'\d+|,|est|ouest','').str.strip()
dummy['address'] = dummy['address'].str.replace(r'-',' ')
dummy['address'] = dummy['address'].str.replace(r'ave','av ')
dummy['address'] = dummy['address'].str.replace(r'av.','av ')
dummy['address'] = dummy['address'].str.replace(r'avnue','av ')
dummy['address'] = dummy['address'].str.replace(r'av ','')
dummy['address'] = dummy['address'].str.replace(r'boulevard','boul ')
dummy['address'] = dummy['address'].str.replace(r'boul.','boul ')
dummy['address'] = dummy['address'].str.replace(r'b. ','boul ')
dummy['address'] = dummy['address'].str.replace(r'b ','boul ')
dummy['address'] = dummy['address'].str.replace(r'boulav.','boul ')
dummy['address'] = dummy['address'].str.replace(r'boul ','')
dummy['address'] = dummy['address'].str.replace(r'boulst','boulsaint ')
dummy['address'] = dummy['address'].str.replace(r'st','saint ')
dummy['address'] = dummy['address'].str.replace(r'chemin','ch ')
dummy['address'] = dummy['address'].str.replace(r'ch. ','ch ')
dummy['address'] = dummy['address'].str.replace(r'ch ','')
dummy['address'] = dummy['address'].str.replace(r'côte','cote')
dummy['address'] = dummy['address'].str.replace(r'cotes','cote')
dummy['address'] = dummy['address'].str.replace(r'neiges','neige')
dummy['address'] = dummy['address'].str.replace(r'de |\.| o|rue |nue |les |le |la ',' ')
dummy['address'] = dummy['address'].str.replace(r' +',' ')
dummy['address'] = dummy['address'].str.strip()

df['address'] = dummy['address']
dummy = dummy[ list(dummy.columns[:-1].insert(0,list(dummy.columns)[-1]) )]
grouped = dummy.groupby('address').mean().reset_index()

# grouping by more common categories per address and giving top 3 most popular and missing venues 

In [13]:
columns = ['address','1st most common','2nd most common','3rd most common']
new_sor = pd.DataFrame(columns = columns)
new_sor['address'] = grouped['address']
for idx in range(len(grouped)):
    l = np.sum(grouped.iloc[idx,1:].astype(float).sort_values(ascending = False) != 0)
    if l >=3:
        new_sor.iloc[idx,1:] = grouped.iloc[idx,1:].astype(float).sort_values(ascending = False).index.values[0:3]
    else:
        new_sor.iloc[idx,1:l+1] = grouped.iloc[idx,1:].astype(float).sort_values(ascending = False).index.values[0:l]

In [14]:
def remove_el(x):
    copy = np.array(list(df['categories'].value_counts().index))
    for el in x:
        if type(el) != type(''):
            continue
        copy = copy[copy != el]
    return copy[:3]

In [15]:
new_sor['3 most popular missing'] = np.nan
for c,i in enumerate(new_sor.values):
    new_sor['3 most popular missing'].iloc[c] = remove_el(i[1:-1].tolist())

In [16]:
new_sor.head(5)

,address,1st most common,2nd most common,3rd most common,3 most popular missing
0,a sherbrooke saint w,Sushi Restaurant,NaN,NaN,"[Café, Park, Residential Building (Apartment /..."
1,aire arzens nord autoroute a,Sandwich Place,NaN,NaN,"[Café, Park, Residential Building (Apartment /..."
2,atwater,Liquor Store,Market,NaN,"[Café, Park, Residential Building (Apartment /..."
3,autoroute a,Gas Station,NaN,NaN,"[Café, Park, Residential Building (Apartment /..."
4,autoroute deux mers,Rest Area,NaN,NaN,"[Café, Park, Residential Building (Apartment /..."


# Kmean clustering

In [17]:
from sklearn.cluster import KMeans

clustered_grouping = grouped.drop('address',axis =1)
kmeans = KMeans(3,random_state = 42).fit(clustered_grouping)

new_sor['labels'] = kmeans.labels_
final_df = df.join(new_sor.set_index('address'), on='address')
final_df = final_df.dropna(subset = ['labels'])

In [18]:
final_df.head(5)

,id,name,categories,lat,lng,address,1st most common,2nd most common,3rd most common,3 most popular missing,labels
1,55aa5627498e9265d39d301f,Camellas Lloret Maison d'hotes,Bed & Breakfast,43.199272,2.141256,l'angle,Bed & Breakfast,NaN,NaN,"[Café, Park, Residential Building (Apartment /...",0.0
2,4ecca289754ac92c121e4d43,Pont-canal de Rébenty,Bridge,43.244132,2.156265,métairie grande,Bridge,NaN,NaN,"[Café, Park, Residential Building (Apartment /...",0.0
3,4ff98791e4b0463f723caa67,Aire de Montréal,Rest Area,43.228609,2.114393,autoroute deux mers,Rest Area,NaN,NaN,"[Café, Park, Residential Building (Apartment /...",0.0
4,59a2b7cc9ec3995174eaf2a2,Total,Gas Station,43.200341,2.140633,du lauragais,Gas Station,NaN,NaN,"[Café, Park, Residential Building (Apartment /...",0.0
10,52a1f001498e09d3f9740621,La Table Saint Crescent,French Restaurant,43.220816,2.206478,du general leclerc,French Restaurant,NaN,NaN,"[Café, Park, Residential Building (Apartment /...",0.0


In [19]:
import folium
#colors = ['red','blue','black','orange','green','darkpurple','lightgray','purple']
colors = ['red','black','green']
mapus = folium.Map(location =[lat,lng] )
for add,label,cat in zip(final_df[['lat','lng']].values, final_df['labels'].astype(int),final_df['categories'] ):
    if cat ==None:
        cat ='Unknown'
    cat = cat.replace("'",'')
    cat = cat.replace('é','e')
    folium.CircleMarker(add, color = colors[label], radius = 2, popup = cat).add_to(mapus)

In [20]:
mapus